# Midterm Exam Big Data part 1
- Name : Adela Krylova
- ID : 5999222017

##1. Task description

Spark Streaming

Create an Apache Spark notebook for handling file stream inside a folder.

Use the data given in news folder (news-1.json, news-2.json, news-3.json)

Put the input inside folder named input-your-student-id. Example: input-51231132

Put the output inside folder named output-your-student-id


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=16205b4c2f0f3bfb780a450822f62cafd9e20ea549a75dc24b090049a2733bdc
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


##2. Create necessary files and directories 

In [3]:
!unzip /content/new.zip

Archive:  /content/new.zip
  inflating: news-3.json             
  inflating: news-2.json             
  inflating: news-1.json             


In [4]:
input = "/content/input-5999222017"
output = "/content/output-5999222017"

##3. Initialize Spark

In [7]:
from pyspark.sql import SparkSession

# creating a new spark session
spark = SparkSession \
    .builder \
    .appName("Spark Streaming") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

### Streaming

In [18]:
#creating a streaming session

spark.conf.set("spark.sql.streaming.schemaInference", True)

streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "off") \
    .option("maxFilesPerTrigger", 1) \
    .load(input)

In [19]:
streaming_df.printSchema()

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



### Writing

In [20]:
writing_df = streaming_df.writeStream \
    .format("json") \
    .option("path", output) \
    .option("checkpointLocation","/content/my_checkpoints") \
    .outputMode("append") \
    .start()

##4. Start Streming

In [22]:
try:
    writing_df.awaitTermination()
except:
    # waiting for manual forcing to end the session
    # showing the output
    out_df = spark.read.json(output)
    out_df.show(truncate=False)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


+---------------------------------------+--------------+----------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|authors                                |category      |date      |headline                                                                                |link                                                                                                             |short_description                                                                                                                                                                                       |
+---------------------------------------+--------------+